互相关运算

In [ ]:
!pip install d2l==0.14

In [7]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):#相当于卷积运算
  """计算二维互相关运算"""
  h,W=K.shape
  Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-W+1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j]=(X[i:i+h,j:j+W]*K).sum()
  return Y

验证上述二维互相关运算的输出

In [8]:
X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积

In [12]:
class Conv2D(nn.Module):
  def __init__(self,kernel_size):
    super().__init__()
    self.weight=nn.Parameter(torch.rand(kernel_size))
    self.bias=nn.Parameter(torch.zeros(1))
  
  def forword(self,X):
    return corr2d(X,self.weight)+self.bias


卷积层的一个简单应用：检测图像中不同颜色的边缘

In [14]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [18]:
K=torch.tensor([[1.0,-1.0]])

输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [19]:
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只可以检测垂直边缘

In [20]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由X生成Y的卷积核

In [21]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))

for i in range(10):
  Y_hat=conv2d(X)
  l=(Y_hat-Y)**2
  conv2d.zero_grad()
  l.sum().backward()
  conv2d.weight.data[:]-=3e-2*conv2d.weight.grad
  if (i+1)%2==0:
    print(f'batch {i+1},loss {l.sum():.3f}')

batch 2,loss 11.168
batch 4,loss 3.448
batch 6,loss 1.223
batch 8,loss 0.469
batch 10,loss 0.187


所学的卷积核的权重张量

In [22]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9446, -1.0330]])